In [44]:
import pandas as pd
df = pd.read_csv("benchmarks/train.csv")

/var/folders/fy/__8z8cpn6gs04465sq9g1nq80000gn/T/ipykernel_8907/1654667150.py:2: DtypeWarning: Columns (14,15) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("benchmarks/train.csv")


In [45]:
import warnings
from typing import Dict, List, Literal, Optional, Tuple, Union
from warnings import warn

import numpy as np
import pandas as pd
import sklearn
from pandas._libs.tslibs.parsing import guess_datetime_format
from pandas.core.dtypes.base import ExtensionDtype
from sklearn import __version__ as sklearn_version
from sklearn.base import TransformerMixin, clone
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.utils.deprecation import deprecated
from sklearn.utils.validation import check_is_fitted

from dirty_cat import DatetimeEncoder, GapEncoder
from dirty_cat._utils import parse_version

# Required for ignoring lines too long in the docstrings
# flake8: noqa: E501


def _infer_date_format(date_column: pd.Series, n_trials: int = 100) -> Optional[str]:
    """Infer the date format of a date column,
    by finding a format which should work for all dates in the column.

    Parameters
    ----------
    date_column : :class:`~pandas.Series`
        A column of dates, as strings.
    n_trials : int, default=100
        Number of rows to use to infer the date format.

    Returns
    -------
    Optional[str]
        The date format inferred from the column.
        If no format could be inferred, returns None.
    """
    if len(date_column) == 0:
        return
    date_column_sample = date_column.dropna().sample(
        frac=min(n_trials / len(date_column), 1), random_state=42
    )
    # try to infer the date format
    # see if either dayfirst or monthfirst works for all the rows
    with warnings.catch_warnings():
        # pandas warns when dayfirst is not strictly applied
        warnings.simplefilter("ignore")
        date_format_monthfirst = date_column_sample.apply(
            lambda x: guess_datetime_format(x)
        )
        date_format_dayfirst = date_column_sample.apply(
            lambda x: guess_datetime_format(x, dayfirst=True),
        )
        print(date_format_monthfirst)
        print(date_format_dayfirst)
    # if one row could not be parsed, return None
    if date_format_monthfirst.isnull().any() or date_format_dayfirst.isnull().any():
        return
    # even with dayfirst=True, monthfirst format can be inferred
    # so we need to check if the format is the same for all the rows
    elif date_format_monthfirst.nunique() == 1:
        # one monthfirst format works for all the rows
        # check if another format works for all the rows
        # if so, raise a warning
        if date_format_dayfirst.nunique() == 1:
            # check if monthfirst and dayfirst haven't found the same format
            if date_format_monthfirst.iloc[0] != date_format_dayfirst.iloc[0]:
                warnings.warn(
                    f"""
                    Both {date_format_monthfirst.iloc[0]} and {date_format_dayfirst.iloc[0]} are valid
                    formats for the dates in column {date_column.name}.
                    Format {date_format_monthfirst.iloc[0]} will be used.
                    """,
                    UserWarning,
                    stacklevel=2,
                )
        return date_format_monthfirst.iloc[0]
    elif date_format_dayfirst.nunique() == 1:
        # only this format works for all the rows
        return date_format_dayfirst.iloc[0]
    else:
        # more than two different formats were found
        # TODO: maybe we could deal with this case
        return


In [46]:
pd.__version__

'2.0.0'

In [47]:
guess_datetime_format("12/1/22")

In [48]:
df["HIREDT"]

0         12/1/22
1          2/3/05
2          2/1/20
3         10/7/99
4         10/7/99
           ...   
112588     1/1/23
112589     1/1/23
112590     1/1/23
112591     1/1/23
112592    1/30/23
Name: HIREDT, Length: 112593, dtype: object

In [49]:
pd.to_datetime(df["HIREDT"])

/var/folders/fy/__8z8cpn6gs04465sq9g1nq80000gn/T/ipykernel_8907/3117163595.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  pd.to_datetime(df["HIREDT"])


0        2022-12-01
1        2005-02-03
2        2020-02-01
3        1999-10-07
4        1999-10-07
            ...    
112588   2023-01-01
112589   2023-01-01
112590   2023-01-01
112591   2023-01-01
112592   2023-01-30
Name: HIREDT, Length: 112593, dtype: datetime64[ns]

In [50]:
guess_datetime_format("30/01/23")

In [51]:
pd.to_datetime(df["HIREDT"])

/var/folders/fy/__8z8cpn6gs04465sq9g1nq80000gn/T/ipykernel_8907/3117163595.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  pd.to_datetime(df["HIREDT"])


0        2022-12-01
1        2005-02-03
2        2020-02-01
3        1999-10-07
4        1999-10-07
            ...    
112588   2023-01-01
112589   2023-01-01
112590   2023-01-01
112591   2023-01-01
112592   2023-01-30
Name: HIREDT, Length: 112593, dtype: datetime64[ns]

In [52]:
print(_infer_date_format(df["HIREDT"]))

21229     None
11040     None
82591     None
55539     None
35412     None
          ... 
24263     None
17713     None
85486     None
108109    None
50550     None
Name: HIREDT, Length: 100, dtype: object
21229     None
11040     None
82591     None
55539     None
35412     None
          ... 
24263     None
17713     None
85486     None
108109    None
50550     None
Name: HIREDT, Length: 100, dtype: object
None


In [53]:
from dirty_cat import TableVectorizer
from dirty_cat.datasets import fetch_traffic_violations

tb = TableVectorizer()
tb.fit_transform(df)

Fit GapEncoderColumn
init ngrams count vectorizer
fit ngrams count vectorizer
fit ngrams count vectorizer done
fit ngrams count vectorizer done2
fit ngrams count vectorizer done3
Batch 880/880 - W change: 0.1276
Tolerance:  0.0001
Epoch 1/5 - Time: 7.52s


KeyboardInterrupt: 

In [ ]:
tb

TableVectorizer()

In [ ]:
from dirty_cat import TableVectorizer
from dirty_cat.datasets import fetch_traffic_violations

ds = fetch_traffic_violations()
#sv = TableVectorizer()
#sv.fit(ds.X)  # This will take a while...

#print(sv.transformers)

/Users/leo/PycharmProjects/dirty_cat_forks/dirty_cat/dirty_cat/datasets/_fetching.py:626: DtypeWarning: Columns (25) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(info["path"], **read_csv_kwargs)


In [ ]:
from dirty_cat import GapEncoder

gap = GapEncoder(n_jobs=1, batch_size=512, hashing=False)

In [ ]:
high_cardinality_cols = ["seqid", "description", "location", "search_reason_for_stop", "state", "make", "model", "charge", "driver_city", "driver_state", "dl_state", "geolocation"]
#high_cardinality_cols = ["description", "location", "search_reason_for_stop", "state", "make", "model", "charge", "driver_city", "driver_state", "dl_state", "geolocation"]
#high_cardinality_cols = ["seqid"]



In [ ]:
data = ds.X[["seqid"]]
# only keep the first 10000 rows
data = data[:10000]

In [ ]:
import numpy as np
data = np.arange(10000).reshape(-1, 1).astype(str)

In [ ]:
gap.fit(data)

Fit GapEncoderColumn
init ngrams count vectorizer
fit ngrams count vectorizer
fit ngrams count vectorizer done
fit ngrams count vectorizer done2
fit ngrams count vectorizer done3
Batch 20/20 - W change: 0.1125
Tolerance:  0.0001
Epoch 1/5 - Time: 2.18s
Batch 20/20 - W change: 0.0876
Tolerance:  0.0001
Epoch 2/5 - Time: 1.56s
Batch 20/20 - W change: 0.0802
Tolerance:  0.0001
Epoch 3/5 - Time: 1.37s
Batch 20/20 - W change: 0.0781
Tolerance:  0.0001
Epoch 4/5 - Time: 1.35s
Batch 20/20 - W change: 0.0776
Tolerance:  0.0001
Epoch 5/5 - Time: 1.14s


GapEncoder(batch_size=512, n_jobs=1)

In [ ]:
gap.fit(data)

Fit GapEncoderColumn
Score: 37844715.045767166
Batch 782/782 - W change: 0.0171
Tolerance:  0.0001
Score: 19796404.739312764
Epoch 1/5 - Time: 11.43s
Batch 782/782 - W change: 0.0171
Tolerance:  0.0001
Score: 19789466.194309905
Epoch 2/5 - Time: 11.21s
Batch 782/782 - W change: 0.0171
Tolerance:  0.0001
Score: 19719531.34417127
Epoch 3/5 - Time: 11.07s


KeyboardInterrupt: 

In [ ]:
# comments
# init var takes ~ 1 min, we should go to hashing

# easily parallelizable

In [ ]:
# check how many cpus are available
import joblib
joblib.cpu_count()

8

In [ ]:
from dirty_cat import MinHashEncoder

In [ ]:
 enc = MinHashEncoder(n_components=5)

In [ ]:
X1 = [['paris, FR'], ['Paris'], ['London, UK'], ['London']]
X2 = [['paris, FR'], ['Paris'], ['London, UK'], ['London']]
import numpy as np
X = np.concatenate((X1, X2), axis=1)


In [ ]:
X.shape

(4, 2)

In [ ]:
X

array([['paris, FR', 'paris, FR'],
       ['Paris', 'Paris'],
       ['London, UK', 'London, UK'],
       ['London', 'London']], dtype='<U10')

In [ ]:
import pandas as pd
pd.DataFrame(X)

,0
0,"paris, FR"
1,Paris
2,"London, UK"
3,London


In [ ]:
enc.fit(X)

MinHashEncoder(n_components=5)

In [ ]:
enc.transform(X).shape

(4, 10)

In [ ]:
ds.X["seqid"]

0          fdcc1a6b-4854-4cde-bb60-248f478fa5b6
1          842dad60-5edf-47a8-9e94-c7e6da729498
2          4db837cc-f2fa-4a5b-9ac8-37698492b5f9
3          79761295-50f6-4336-8b48-fdf55e87a326
4          f9a7a508-386c-466e-95b2-dbf26d7d59fe
                           ...                 
1578149    b1463ded-4b1a-4e7f-ada9-9211bbdc1cbb
1578150    b1463ded-4b1a-4e7f-ada9-9211bbdc1cbb
1578151    b1463ded-4b1a-4e7f-ada9-9211bbdc1cbb
1578152    b1463ded-4b1a-4e7f-ada9-9211bbdc1cbb
1578153    845c2732-d70b-42ef-bde8-603f9563aa28
Name: seqid, Length: 1578154, dtype: object